In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from time import time

In [18]:
words = pd.read_csv('/home/kopita/data_sets/nytimes/vocab.nytimes.txt', header=None)
words.columns = ['word']

In [3]:
df = pd.read_csv('/home/kopita/data_sets/nytimes/docword.nytimes.txt', sep=' ', skiprows=3,
                 header=None)
df.columns = ['docID','wordID','count']
dff = df.values

In [4]:
rows, row_pos = np.unique(dff[:, 0], return_inverse=True)
cols, col_pos = np.unique(dff[:, 1], return_inverse=True)
pivot_table = sps.coo_matrix((dff[:, 2], (row_pos, col_pos)),
                             shape=(len(rows), len(cols)))
pivot_table = pivot_table.tocsr()
pivot_table.sort_indices()

In [396]:
indices, counts = np.unique(pivot_table.indices, return_counts=True)

In [402]:
np.argsort(counts)

array([80514, 62630, 62533, ..., 25710, 27948, 33454])

In [408]:
counts[33454]

108622

In [387]:
pivot_table.indptr[:100]

array([    0,   128,   236,   502,   671,   784,   962,  1308,  1627,
        2149,  2264,  2439,  2526,  2650,  2851,  3212,  3528,  3844,
        4137,  4474,  4628,  4810,  4814,  5200,  5436,  5707,  5909,
        6167,  6273,  6511,  6749,  7055,  7361,  7557,  7753,  7947,
        8141,  8269,  8397,  8728,  9059,  9320,  9581,  9741,  9901,
       10137, 10373, 10482, 10591, 10832, 11073, 11263, 11453, 11693,
       11933, 12044, 12155, 12395, 12635, 12765, 12895, 13177, 13459,
       13715, 13981, 14211, 14438, 14746, 14933, 15173, 15413, 15524,
       15635, 15909, 16069, 16260, 16503, 16633, 17019, 17136, 17316,
       17463, 17577, 17727, 17851, 18010, 18181, 18584, 18986, 19227,
       19504, 19721, 19980, 20236, 20508, 20607, 20791, 21061, 21314, 21558], dtype=int32)

In [373]:
pivot_table.indptr.shape

(299753,)

In [391]:
pivot_table[:,:10].sum(axis=0)

matrix([[  70,   25,   53,  529,   29,   47,  107, 3950, 7331, 1591]], dtype=int64)

In [409]:
np.count_nonzero(pivot_table[:,33454].toarray())

108622

In [377]:
pivot_table

<299752x101636 sparse matrix of type '<class 'numpy.int64'>'
	with 69679427 stored elements in Compressed Sparse Row format>

In [393]:
#pivot_table[:,:10]
#pivot_table.indices.reshape([299752,101636])
pivot_table.indices[:1000]

array([   412,    533,   2338,   2803,   3056,   3067,   3291,   3353,
         4053,   4927,   5251,   6884,   6942,   6970,   7292,   7398,
         7401,   7405,   7540,   7786,   9081,   9381,   9955,   9979,
        10122,  10470,  10783,  11758,  12605,  12956,  13354,  13987,
        14250,  14748,  14810,  14847,  15498,  15708,  15881,  16248,
        16380,  16576,  16738,  16847,  17649,  17655,  17815,  18067,
        18195,  18348,  18561,  18698,  18984,  22120,  22121,  22140,
        22284,  22306,  22864,  23497,  24478,  24847,  25598,  25710,
        25711,  25939,  26101,  26391,  28038,  28394,  28395,  29149,
        29158,  29345,  29591,  30661,  30995,  31422,  31568,  31570,
        31727,  31730,  31925,  32539,  33005,  33454,  33928,  34444,
        34479,  34544,  34765,  34873,  35472,  35476,  35523,  37885,
        37942,  38036,  39105,  39120,  39125,  39330,  40216,  40365,
        40459,  41035,  41042,  41140,  42505,  42839,  43547,  43786,
      

In [385]:
pivot_table.nonzero

<bound method spmatrix.nonzero of <299752x101636 sparse matrix of type '<class 'numpy.int64'>'
	with 69679427 stored elements in Compressed Sparse Row format>>

In [346]:
pivot_table.getcol(0).toarray()

array([[0],
       [0],
       [0],
       ..., 
       [0],
       [0],
       [0]])

In [325]:
len(pivot_table.getrow(1002).indices)

198

In [258]:
pivot_table[-10:,:22].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [259]:
q[-10:,:22].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [209]:
col_count = []
for n_col in range(pivot_table.shape[1]):
    indices = len(pivot_table.getcol(n_col).indices)
    col_count.append(indices)


KeyboardInterrupt: 

In [ ]:
col_count

In [ ]:
top_indices_i = np.argsort(top_indices)[-10:][::-1]

In [206]:
len(pivot_table.getcol(0).indices)

68

In [207]:
np.count_nonzero(pivot_table.getcol(0).toarray())

68

In [197]:
tt = pivot_table.getcol(1)

In [198]:
np.count_nonzero(tt.toarray())

25

In [201]:
pivot_table

<299752x101636 sparse matrix of type '<class 'numpy.int64'>'
	with 69679427 stored elements in Compressed Sparse Row format>

In [97]:
qq = pivot_table.getrow(3)
qq

<1x101636 sparse matrix of type '<class 'numpy.int64'>'
	with 169 stored elements in Compressed Sparse Row format>

In [105]:
test = []
for n_row in range(pivot_table.shape[0]):
    indices = pivot_table.getrow(n_row).indices
    test.append(len(indices))

In [139]:
col_pos

array([  412,   533,  2338, ..., 69437, 80303, 81691])

In [177]:
np.sum(pivot_table[128, :].toarray())

536

In [172]:
np.argsort(test)[-10:][::-1]

array([122633, 299001, 299044, 298574, 298579, 122401, 231542, 231518,
       177316, 185007])

In [155]:
len(test)

299752

In [103]:
len(qq.indices)

169

In [88]:
np.argmax(qq.data)

124

In [5]:
true_k = 20
print("Extracting features from the training dataset using a sparse vectorizer")
t0 = time()
vectorizer = TfidfTransformer()
X = vectorizer.fit_transform(pivot_table)
print("done in %.2fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

Extracting features from the training dataset using a sparse vectorizer
done in 3.42s
n_samples: 299752, n_features: 101636



In [200]:
vectorizer.

{'norm': 'l2', 'smooth_idf': True, 'sublinear_tf': False, 'use_idf': True}

In [9]:
svd = TruncatedSVD(100)
nX = svd.fit_transform(X)

In [13]:
svd.explained_variance_ratio_.sum()

0.11475546253294396

In [24]:
words.shape

(102660, 1)

In [ ]:
svd = TruncatedSVD(1000)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

nX = lsa.fit_transform(X)

In [30]:
X

<299752x101636 sparse matrix of type '<class 'numpy.float64'>'
	with 69679427 stored elements in Compressed Sparse Row format>

In [28]:
words.values[0]

array(['aah'], dtype=object)

In [47]:
vectorizer = TfidfTransformer()

In [37]:
pivot_table = np.zeros((len(rows), len(cols)))

MemoryError: 

In [ ]:
data = np.array([[ 4057,     8,  1374],
                 [ 4057,     9,   759],
                 [ 4057,    11,    96],
                 [89205,    16,   146],
                 [89205,    17,   154],
                 [89205,    18,   244],
                 [ 4057,    11,     4]])

rows, row_pos = np.unique(data[:, 0], return_inverse=True)
cols, col_pos = np.unique(data[:, 1], return_inverse=True)

pivot_table = np.zeros((len(rows), len(cols)), dtype=data.dtype)
pivot_table[row_pos, col_pos] = data[:, 2]
>>> pivot_table # the element at [0, 2] should be 100!!!
array([[1374,  759,    4,    0,    0,    0],
       [   0,    0,    0,  146,  154,  244]])

import scipy.sparse as sps
pivot_table = sps.coo_matrix((data[:, 2], (row_pos, col_pos)),
                             shape=(len(rows), len(cols))).A
>>> pivot_table # now repeated elements are added together
array([[1374,  759,  100,    0,    0,    0],
       [   0,    0,    0,  146,  154,  244]])

In [13]:
df.head()

,docID,wordID,count
0,1,413,1
1,1,534,1
2,1,2340,1
3,1,2806,1
4,1,3059,1


In [ ]:
df.pivot(index='docID', columns='wordID', values='count')

In [ ]:
df.head()

In [ ]:
data = np.loadtxt('/home/kopita/data_sets/nytimes/docword.nytimes.txt', dtype=int, delimiter=' ', skiprows=3)

In [ ]:
df.head()

In [ ]:
(range(2))

In [ ]:
df_p = df.pivot(index='date', columns='variable', values='value')

In [ ]:
tfid_t = TfidfTransformer()
tfid = tfid_t.fit_transform(df)

In [ ]:
n_components = 1000
svd = TruncatedSVD(n_components)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(X)

In [ ]:
tfid

In [ ]:
df.head()

In [ ]:
tf

In [ ]:
data = numpy.loadtxt(open(filename,'rb'),delimiter=csv_delimiter,skiprows=0)
    return data